In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [13]:
r = requests.get("http://www.amazon.com/16x24-Katherine-Gendreau-Cannon-Beach/dp/B00MBQUI4O/ref=sr_1_124?s=art&ie=UTF8&qid=1462469430&sr=1-124")
# going to the website

In [14]:
r
# A 200 response is GOOD, anything else is bad

<Response [200]>

In [15]:
r.url.split('/')[-1]

u'B00MBQUI4O'

In [16]:
b = BeautifulSoup(r.text, 'html.parser')
# this is a python object that parses the html code

In [25]:
# get the creator
b.find('a', {'id':'fine-ART-ProductLabelArtistNameLink'}).text

u'Leonard Freed'

In [12]:
# get the title
b.find('span', {'id': 'fineArtTitle'}).text

u'Mardi Gras,'

In [13]:
# get the price
price = b.find('span', {'id': 'priceblock_ourprice'}).text.replace('$','').replace(',', '')

if '.' in price:
    price = float(price.replace('.', ''))/100
else:
    price = float(price)
price

45000.0

In [14]:
re.findall('http://ecx.images-amazon.com/images/I/.*\.jpg', r.text)[0]

u'http://ecx.images-amazon.com/images/I/51GvBhnt4sL._QL70_.jpg'

In [17]:
b.findAll('span', {'class': 'a-size-base'})

[<span class="a-size-base" id="mnbaProductLabelMediaType">C-Print</span>,
 <span class="a-size-base" id="mnbaProductLabelEditionInfo">Edition 1-50 of 50</span>,
 <span class="a-size-base">16x24 in.</span>,
 <span class="a-size-base">24x36 in.</span>,
 <span class="a-size-base a-color-base">&amp; <b>FREE Shipping</b></span>,
 <span class="a-carousel-pagination a-size-base"><span class="a-carousel-page-count"><span class="a-carousel-page-current">1</span> <span class="a-carousel-page-max">1</span></span><span class="a-carousel-restart-container"><span class="a-text-separator"></span><a class="a-carousel-restart" href="#"></a></span><span aria-live="polite" class="a-carousel-accessibility-page-info a-offscreen">1 1</span></span>,
 <span class="a-carousel-pagination a-size-base"><span class="a-carousel-page-count"><span class="a-carousel-page-current">1</span> <span class="a-carousel-page-max">1</span></span><span class="a-carousel-restart-container"><span class="a-text-separator"></span><

In [24]:
b.findAll('span', {'class': "a-size-medium a-color-price"})

[<span class="a-size-medium a-color-price" id="priceblock_ourprice">$200.00</span>]

In [15]:
# get the size
size = [a.strip() for a in b.find('span', {'id': 'mnba_buybox_size'}).text.split('x')]

if len(size) == 2:
    height = float(size[0].replace('in.','').strip())
    width = float(size[1].replace('in.','').strip())
    depth = 1.
elif len(size) == 3:
    height = float(size[0].replace('in.','').strip())
    width = float(size[1].replace('in.','').strip())
    depth = float(size[2].replace('in.','').strip())

print(height, width, depth)
sq_inches = height*width

(41.0, 61.0, 1.0)


In [19]:
size = b.find('span', {'id': 'mnba_buybox_size'}).text
size

u'41 x 61 in.'

In [ ]:
#get size (height and width)
type(b.findAll('th', {'class': 'a-span5 a-size-base'}))
#b.findAll('td', {'class': 'a-span7 a-size-base'})

In [ ]:
#get type
b.find('span', {'id': 'mnbaProductLabelMediaType'}).text

In [68]:
#get artwork description
b.find('div', {'id': 'productDescription_feature_div'}).getText().replace('\n','')

u'Fire Hydrant, Harlem, 1963, 20x16, Archival Iris Print, signed (recto) by the photographer Leonard Freed. Edition number: 22/100. Only 25 copies were printed of the planned edition of 100. Certificate of authenticity.'

In [65]:
for desc, value in zip(b.findAll('th', {'class':'a-span5 a-size-base'}), b.findAll('td', {'class':'a-span7 a-size-base'})):
    if desc.text.strip() in ['Height', 'Width']:
        print value.text.strip()

20 inches
16 inches


In [ ]:
def scrapePainting(url):
    r = requests.get(url)
    b = BeautifulSoup(r.text, 'html.parser')
    url_id = r.url.split('/')[-1]
    artist = b.find('a', {'id':'fine-ART-ProductLabelArtistNameLink'}).text
    title = b.find('span', {'id': 'fineArtTitle'}).text
    image = re.findall('http://ecx.images-amazon.com/images/I/.*\.jpg', r.text)[0]
    price = price = b.find('span', {'id': 'priceblock_ourprice'}).text.replace('$','').replace(',', '')
    if '.' in price:
            price = float(price.replace('.', ''))/100
    else:
            price = float(price)
    desc = b.find('div', {'id': 'productDescription_feature_div'}).text
    size = [a.strip() for a in b.find('span', {'id': 'mnba_buybox_size'}).text.split('x')]
    if len(size) == 2:
        height = float(size[0].replace('in.','').strip())
        width = float(size[1].replace('in.','').strip())
        depth = 1.
    elif len(size) == 3:
        height = float(size[0].replace('in.','').strip())
        width = float(size[1].replace('in.','').strip())
        depth = float(size[2].replace('in.','').strip())

    return { 'url_id': url_id, 'artist': artist, 'title':title, 'image': image, 'price': price, 'description':desc, 'height': height, 'width': width}

In [ ]:
import pandas as pd
paintings = [scrapePainting("http://www.amazon.com/Dream-of-Migrants/dp/B00H9PNMIQ")]
pd.DataFrame(paintings)

In [ ]:
paintings = []
    
for page in range(1,401):
    #url = "http://www.amazon.com/s/ref=sr_pg_2?rh=n%3A4991425011%2Cn%3A%214991426011%2Cn%3A6685269011%2Cn%3A6685289011%2Cp_36%3A7175858011&page="+str(page)+"&bbn=6685289011&sort=price-desc-rank&ie=UTF8&qid=1462043758"
    url = "http://www.amazon.com/s/ref=lp_6685289011_pg_2?rh=n%3A4991425011%2Cn%3A%214991426011%2Cn%3A6685269011%2Cn%3A6685289011&page=re&ie=UTF8&qid=1462071400"
    r = requests.get(url)
    r.text
    for painting in 
    

In [ ]:
r = requests.get('http://www.amazon.com/s/ref=lp_6685289011_pg_2?rh=n%3A4991425011%2Cn%3A%214991426011%2Cn%3A6685269011%2Cn%3A6685289011&page=1&ie=UTF8&qid=1462071400')
b = BeautifulSoup(r.text, 'html.parser')

url_1 = []
for link in b.findAll('a', {'class':"a-link-normal a-text-normal"}):
    #print(link.get('href'))
    url_1.append(link.get('href'))
    
url_list = url_1[::2]

url_list

In [35]:
desc = b.find('div', {'id': 'productDescription_feature_div'})
if desc is not None:     
    desc = b.find('div', {'id': 'productDescription_feature_div'}).getText()

In [36]:
print desc



Fire Hydrant, Harlem, 1963, 20x16, Archival Iris Print, signed (recto) by the photographer Leonard Freed. Edition number: 22/100. Only 25 copies were printed of the planned edition of 100. Certificate of authenticity.




In [37]:
type(desc)

unicode

In [38]:
desc is not None

True

In [48]:
b.find('div', {'id': 'productDescription_feature_div'}).getText()

u'\n\nFire Hydrant, Harlem, 1963, 20x16, Archival Iris Print, signed (recto) by the photographer Leonard Freed. Edition number: 22/100. Only 25 copies were printed of the planned edition of 100. Certificate of authenticity.\n\n'

In [61]:
image = re.findall('http://ecx.images-amazon.com/images/I/.*\.jpg', r.text)
if image is not None:
    image = re.findall('http://ecx.images-amazon.com/images/I/.*\.jpg', r.text)

In [62]:
print image

[u'http://ecx.images-amazon.com/images/I/51hNjtVjR3L._QL70_.jpg', u'http://ecx.images-amazon.com/images/I/51hNjtVjR3L._QL70_.jpg', u'http://ecx.images-amazon.com/images/I/51hNjtVjR3L._QL70_.jpg" data-old-hires="" onload="this.onload=\'\';setCSMReq(\'af\');setCSMReq(\'cf\');" class="a-dynamic-image a-stretch-horizontal" id="fine-art-landingImage" data-a-dynamic-image="{&quot;http://ecx.images-amazon.com/images/I/51hNjtVjR3L.jpg', u'http://ecx.images-amazon.com/images/I/51hNjtVjR3L._SS58_.jpg', u'http://ecx.images-amazon.com/images/I/51hNjtVjR3L._SS58_.jpg","large":"http://ecx.images-amazon.com/images/I/51hNjtVjR3L.jpg","main":{"http://ecx.images-amazon.com/images/I/51hNjtVjR3L.jpg', u'http://ecx.images-amazon.com/images/I/01IEknlRWxL.jpg']
